In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import callbacks
from uproot_io import Events, View
from tensorflow.keras.models import load_model

In [3]:
# the data
events = Events("/Users/mohammed/code/neutrinos-dune-files/CheatedRecoFile_1.root")

In [4]:
event_numbers = np.array(list(set(events.event_number)))
len(event_numbers)

9310

In [5]:
def oneHotLabelling():
    label = []
    
    for n in event_numbers:
        mcp = []
        index = np.where(events.event_number == n)[0]

        for i in index:
            parent = events.reco_parent_index[i]
            if parent == 0:
                mcp.append(events.mc_pdg[i])

        if 11 in mcp or -11 in mcp:
            label.append([0,1,0])    
        
        elif 13 in mcp or -13 in mcp:
            label.append([1,0,0])
            
        else:
            label.append([0,0,1])
            
    return np.array(label).astype(float)

In [6]:
one_hot_labels = oneHotLabelling()
one_hot_labels

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [7]:
view_w = View(events, "w")

In [8]:
#image creation 2

def imagenew():
    
    images = []
    
    for n in event_numbers:
    
        x = view_w.x[n]
        z = view_w.z[n]
        adc = view_w.adc[n]
        vx = view_w.true_vtx_x[n]
        vz = view_w.true_vtx_z[n]

        matrix_size = 256

        matrix, xedges, yedges = np.histogram2d(x, z, bins=matrix_size, range=[[np.floor(np.mean(x))-150, np.floor(np.mean(x))+150], [np.floor(np.mean(z))-150, np.floor(np.mean(z))+150]], weights=adc)
        matrix = (matrix > 0).astype(int)

        matrix = np.floor(matrix).astype(np.uint8).reshape(256, 256, 1)
        images.append(matrix)
    
    return np.stack(images)

In [9]:
images = imagenew()

In [10]:
random_numbersold = np.random.randint(0,max(event_numbers) , max(event_numbers))
seventy = int(0.7*len(random_numbersold))
training = random_numbersold[:seventy]
testing = random_numbersold[seventy:]

In [11]:
training = np.load("training_events.npy")
testing = np.load("testing_events.npy")

In [12]:
x_train = images[training]
y_train = one_hot_labels[training]

x_test = images[testing]
y_test = one_hot_labels[testing]

In [13]:
len(training), len(testing)

(6445, 2763)

In [14]:
num_classes = 3

In [38]:
# Define our simplest model first
input_layer = keras.layers.Input(x_train[0].shape)
x = keras.layers.Conv2D(16, (16,16), activation='relu')(input_layer) # Replace None with a 2D convolution with 32 filters of size (3,3) and relu activation
x = keras.layers.MaxPooling2D((2,2))(x) # Replace None with a MaxPooling2D layer to downsample by a factor of 2 in both dimensions
x = keras.layers.Dropout(0.25)(x) # Replace None with a droput layer with a fraction of 0.25
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(num_classes, activation='softmax')(x) # Replace None with a final dense output layer with num_classes neurons and softmax activation
cnn_model = keras.Model(input_layer, x)
cnn_model.summary()

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 256, 256, 1)]     0         
                                                                 
 conv2d_18 (Conv2D)          (None, 241, 241, 16)      4112      
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 120, 120, 16)      0         
 ng2D)                                                           
                                                                 
 dropout_23 (Dropout)        (None, 120, 120, 16)      0         
                                                                 
 flatten_12 (Flatten)        (None, 230400)            0         
                                                                 
 dense_33 (Dense)            (None, 3)                 691203    
                                                          

In [39]:
# The batch size controls the number of images that are processed simultaneously
batch_size = 64
# The number of epochs that we want to train the network for - model 2, 20 is best
epochs = 5
# The learning rate (step size in gradient descent)
learning_rate = 0.001

In [40]:
# Define the loss function - for a multi-class classification task we need to
# use categorical crossentropy loss
loss_function = keras.losses.categorical_crossentropy
# The optimiser performs the gradient descent for us. There are a few different
# algorithms, but Adam is one of the more popular ones
optimiser = keras.optimizers.legacy.Adam(learning_rate=learning_rate)
# Now we compile the model with the loss function and optimiser
model.compile(loss=loss_function, optimizer=optimiser, metrics=['accuracy'])

In [41]:
# Train the model using the training data with the true target outputs.
# Fill in the required arguments using the clues given above
# with tf.device('/cpu:0'):
history = model.fit(x = x_train, y = y_train, batch_size = batch_size, epochs = epochs,
                  validation_data = (x_test, y_test), verbose = 1)

Epoch 1/5
 20/101 [====>.........................] - ETA: 2:35 - loss: 0.9774 - accuracy: 0.6367

KeyboardInterrupt: 

In [ ]:
# # model 2

# from tensorflow import keras
# from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# # Define the input shape
# input_shape = x_train[0].shape  # Replace with the shape of your input data

# # Input layer
# input_layer = keras.layers.Input(x_train[0].shape)
# # Convolutional layers
# x = Conv2D(32, (7, 7), activation='relu', padding='same')(input_layer)
# x = MaxPooling2D((2, 2))(x)
# x = Conv2D(64, (5, 5), activation='relu', padding='same')(x)
# x = MaxPooling2D((2, 2))(x)
# x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
# x = MaxPooling2D((2, 2))(x)

# # Flatten the output of the conv layers
# x = Flatten()(x)
# # Dense layer
# x = Dense(128, activation='relu')(x)
# x = Dropout(0.5)(x)
# # Output layer
# output = Dense(num_classes, activation='softmax')(x)  # 3 classes for the output

# # Creating the model
# model = keras.Model(inputs=input_layer, outputs=output)
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# model.summary()